In [ ]:
!pip install pandas numpy matplotlib gensim nltk tqdm xgboost scikit-learn


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import zipfile
import os

# Specify the path where the ZIP file is uploaded
zip_file_path = 'data.zip'  # Update with the correct file name if needed

# Extract ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content')  # Extract to /content directory

# Check if the folders are extracted correctly
os.listdir('/content')



['.config', 'data.zip', 'data', 'sample_data']

In [ ]:
pos_dir = '/content/data/pos/'
neg_dir = '/content/data/neg/'

# Get all .txt files from each folder
pos_files = [os.path.join(pos_dir, file) for file in os.listdir(pos_dir) if file.endswith('.txt')]
neg_files = [os.path.join(neg_dir, file) for file in os.listdir(neg_dir) if file.endswith('.txt')]

# Verify the number of files
print(f"Number of positive reviews: {len(pos_files)}")
print(f"Number of negative reviews: {len(neg_files)}")


Number of positive reviews: 12500
Number of negative reviews: 12500


In [ ]:
pos_reviews = []
neg_reviews = []

# Read positive reviews
for file in pos_files:
    with open(file, 'r') as f:
        pos_reviews.append(f.read())

# Read negative reviews
for file in neg_files:
    with open(file, 'r') as f:
        neg_reviews.append(f.read())

# Check the first few reviews
print(f"First positive review: {pos_reviews[0]}")
print(f"First negative review: {neg_reviews[0]}")


First positive review: Probably the first Portuguese film I have seen in my life, and I enjoyed it. The plot is related of how the young army officers took the power in Portugal in 1974, to finally defeat the fascist government of Caetano and to also finalize the wars in the colonies, i.e. Mozambique, Angola, and Guinea (Bissau)- Cape Vert. Most of the events shown in the film reflect with exactitude the behavior of the army officers and soldiers to conduct the coup, of the oppressed people, who were very happy with this new development and the liberty, the resistance of Caetano's men, and also in a subtle way of most conservative officials, including Spinola, who took over as the new president. The Portuguese revolution can be remembered because of the action of several young officers, but for me the most interesting part of the film was when the young captain expressed that Portugal should develop itself democratically, and this is what the country achieved some years after this coup

In [ ]:
import pandas as pd

# Create a DataFrame
labels = ['positive'] * len(pos_reviews) + ['negative'] * len(neg_reviews)
reviews = pos_reviews + neg_reviews

data = pd.DataFrame({'Review': reviews, 'Label': labels})

# Show the first few rows
data.head()


,Review,Label
0,Probably the first Portuguese film I have seen...,positive
1,I just watched Antwone Fisher on BRAVO. What a...,positive
2,in 1976 i had just moved to the us from ceylon...,positive
3,I find this movie very enjoyable. The plot is ...,positive
4,The 12th animated Disney classic is a reasonab...,positive


In [ ]:
data.to_csv('/content/reviews.csv', index=False)


In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/reviews.csv')

# Check the data
print(data.head())



                                              Review     Label
0  Probably the first Portuguese film I have seen...  positive
1  I just watched Antwone Fisher on BRAVO. What a...  positive
2  in 1976 i had just moved to the us from ceylon...  positive
3  I find this movie very enjoyable. The plot is ...  positive
4  The 12th animated Disney classic is a reasonab...  positive


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK datasets
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Function for text preprocessing
def preprocess_text(text):
    # Remove non-alphabetic characters (e.g., punctuation)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()  # Convert to lowercase
    text = text.split()  # Split into words
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]  # Lemmatize and remove stopwords
    return ' '.join(text)

# Apply preprocessing to the reviews
data['Processed_Review'] = data['Review'].apply(preprocess_text)

# Check the first few processed reviews
print(data[['Review', 'Processed_Review']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


                                              Review  \
0  Probably the first Portuguese film I have seen...   
1  I just watched Antwone Fisher on BRAVO. What a...   
2  in 1976 i had just moved to the us from ceylon...   
3  I find this movie very enjoyable. The plot is ...   
4  The 12th animated Disney classic is a reasonab...   

                                    Processed_Review  
0  probably first portuguese film seen life enjoy...  
1  watched antwone fisher bravo awesome movie inc...  
2  moved u ceylon married little three year begin...  
3  find movie enjoyable plot simple easily digest...  
4  th animated disney classic reasonable movie to...  


In [ ]:
from sklearn.model_selection import train_test_split

X = data['Processed_Review']  # Features (processed reviews)
y = data['Label']  # Target (labels: positive or negative)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the splits
print(X_train.shape, X_test.shape)


(20000,) (5000,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 features

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = vectorizer.transform(X_test)

# Check the shape of the resulting feature matrices
print(X_train_tfidf.shape, X_test_tfidf.shape)


(20000, 5000) (5000, 5000)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Check the accuracy of the model
from sklearn.metrics import accuracy_score, classification_report

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")


Accuracy: 0.8824
Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.87      0.88      2485
    positive       0.87      0.90      0.88      2515

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga']  # Solvers to try
}

# Initialize the GridSearchCV with Logistic Regression
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=3, n_jobs=-1)

# Fit the grid search on the training data
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")


Best Parameters: {'C': 1, 'solver': 'saga'}
Best Score: 0.8776500043884695


In [ ]:
# Example test review
test_review = "I am feeling very dull"

# Preprocess the review
processed_review = preprocess_text(test_review)

# Transform the review using the same TF-IDF vectorizer
test_review_tfidf = vectorizer.transform([processed_review])

# Predict the sentiment
predicted_label = model.predict(test_review_tfidf)

print(f"Predicted Sentiment: {predicted_label[0]}")


NameError: name 'preprocess_text' is not defined

In [ ]:
import joblib

# Save the model and vectorizer
joblib.dump(model, 'sentiment_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')



['vectorizer.pkl']

In [ ]:
from google.colab import files

# Download the saved model and vectorizer
files.download('sentiment_model.pkl')
files.download('vectorizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>